In [1]:
%matplotlib inline
import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve,auc
from nltk.stem.porter import PorterStemmer

con= sqlite3.connect('database.sqlite')

filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score !=3""",con)

def partition(x):
    if x<3:
        return 'negative'
    return 'positive'

actualScore=filtered_data['Score']
positiveNegative = actualScore.map(partition)
filtered_data['Score'] = positiveNegative 



In [2]:
filtered_data.shape
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [3]:
display = pd.read_sql_query("""SELECT * FROM Reviews WHERE Score != 3 AND UserId ="AR5J8UI46CURR"ORDER BY ProductId""",con)
display

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [4]:
sorted_data =filtered_data.sort_values('ProductId',axis=0,ascending=True,)

In [5]:
final =sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"},keep='first',inplace=False)
final.shape

(364173, 10)

In [6]:
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100


69.25890143662969

In [7]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]
final.shape

(364171, 10)

In [8]:
print(final.shape)
final['Score'].value_counts()

(364171, 10)


positive    307061
negative     57110
Name: Score, dtype: int64

In [9]:
count_vect = CountVectorizer()
final_counts=count_vect.fit_transform(final['Text'].values)

In [10]:
final_counts.get_shape()

(364171, 115281)

In [11]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

stop =set(stopwords.words('english'))
sno =nltk.stem.SnowballStemmer('english')

def cleanhtml(sentence):
    cleanr= re.compile('<.*?')
    cleantext=re.sub(cleanr,'',sentence)
    return cleantext

def cleanpunc(sentence):
    cleaned=re.sub(r'[?|!|\'|"|#]',r' ',sentence)
    cleaned=re.sub(r'[.|,|)|(|\|/]',r' ',cleaned )
    return cleaned
print(stop)
print(sno.stem('tasty'))
    

{'against', 'were', 'other', 'how', 'are', 'then', 'his', 'before', 'itself', 'same', 'on', 'ma', 'very', 'ain', 'up', 'for', 'in', 'does', 'ourselves', 's', 'only', 't', 'because', "needn't", 'here', 'your', 'if', 'too', 'been', 'she', "shouldn't", 'further', 'doesn', 'out', 'don', 'yourself', 'll', 'until', "won't", 'down', 'y', 'a', 'more', 'what', 'no', 'few', 'with', 'myself', "you're", 'as', 'off', 'our', 'now', 'aren', 'himself', 'than', "mustn't", 'each', 'nor', 'm', 'haven', 'doing', 'below', 'over', 'my', "hasn't", 'own', 'shan', "hadn't", 'once', 'isn', "aren't", 've', 'is', 'why', 'which', 'has', 'will', 'mightn', 'at', 'and', 'shouldn', "shan't", "you'll", 'be', 'their', 'do', 'have', 'had', "you've", 'between', 'hers', 'again', 'just', 'an', 'above', 're', "wouldn't", 'ours', 'we', "should've", "doesn't", 'or', 'into', 'where', 'of', 'that', 'under', "she's", 'themselves', 'didn', 'i', 'd', 'from', 'when', 'weren', 'he', 'was', 'can', 'such', "don't", 'during', 'yours', '

In [12]:
i=0
str1=''
final_string=[]
all_positive_words=[]
all_negative_words=[]
s=''
for sent in final['Text'].values:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha())& (len(cleaned_words)>2)):
                if(cleaned_words.lower() not in stop):
                    s =(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if(final['Score'].values)[i]=='positive':
                        all_positive_words.append(s)
                    if(final['Score'].values)[i]=='negative':
                        all_negative_words.append(s)  
                        
                else:
                    continue
            else:
                continue
    str1=b" ".join(filtered_sentence)            
     
    final_string.append(str1)
    i+=1
                        

In [13]:
final['CleanedText']=final_string

In [14]:
final.head(3)
conn=sqlite3.connect('final.sqlite')


In [15]:
freq_dist_positive =nltk.FreqDist(all_positive_words)
freq_dist_negative =nltk.FreqDist(all_negative_words)
print(freq_dist_positive.most_common(20))
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print(freq_dist_negative.most_common(20))

[(b'like', 139090), (b'tast', 128451), (b'good', 112137), (b'flavor', 109272), (b'love', 107045), (b'use', 103571), (b'great', 102779), (b'product', 99413), (b'one', 95338), (b'tri', 86209), (b'tea', 83721), (b'coffe', 78484), (b'make', 74805), (b'get', 71934), (b'food', 64703), (b'amazon', 57810), (b'would', 55460), (b'time', 55179), (b'buy', 53879), (b'realli', 52545)]
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
[(b'tast', 34471), (b'like', 32270), (b'product', 29498), (b'one', 20411), (b'flavor', 19520), (b'would', 17989), (b'tri', 17673), (b'use', 15272), (b'good', 14960), (b'coffe', 14645), (b'get', 13750), (b'buy', 13686), (b'order', 12838), (b'food', 12730), (b'tea', 11631), (b'amazon', 11247), (b'even', 10975), (b'box', 10829), (b'make', 9812), (b'time', 9784)]


In [16]:
count_vect= CountVectorizer(ngram_range=(1,2))
final_bigram_counts =count_vect.fit_transform(final['Text'].values)

In [17]:
final_bigram_counts.get_shape()

(364171, 2910192)

In [18]:
tf_idf_vect=TfidfVectorizer(ngram_range=(1,2))
final_tf_idf =tf_idf_vect.fit_transform(final['Text'].values)

In [19]:
final_tf_idf.get_shape()

(364171, 2910192)

In [20]:
features =tf_idf_vect.get_feature_names()
len(features)

2910192

In [21]:
print(final_tf_idf[3,:].toarray([0]))

[[0. 0. 0. ... 0. 0. 0.]]


In [22]:
def top_tfidf_feats(row,features,top_n=25):
    topn_ids =np.argsort(row)[::-1][:top_n]
    top_feats =[(features[i],row[i])for i in topn_ids]
    df=pd.DataFrame(top_feats)
    df.columns=['features','tfidf']
    return df
top_tfidf= top_tfidf_feats(final_tf_idf[1,:].toarray()[0],features,25)

In [23]:
top_tfidf

,features,tfidf
0,sendak books,0.173437
1,rosie movie,0.173437
2,paperbacks seem,0.173437
3,cover version,0.173437
4,these sendak,0.173437
5,the paperbacks,0.173437
6,pages open,0.173437
7,really rosie,0.168074
8,incorporates them,0.168074
9,paperbacks,0.168074


In [24]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle

model =KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin',binary=True)





In [25]:

model.wv['computer']


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


array([ 1.07421875e-01, -2.01171875e-01,  1.23046875e-01,  2.11914062e-01,
       -9.13085938e-02,  2.16796875e-01, -1.31835938e-01,  8.30078125e-02,
        2.02148438e-01,  4.78515625e-02,  3.66210938e-02, -2.45361328e-02,
        2.39257812e-02, -1.60156250e-01, -2.61230469e-02,  9.71679688e-02,
       -6.34765625e-02,  1.84570312e-01,  1.70898438e-01, -1.63085938e-01,
       -1.09375000e-01,  1.49414062e-01, -4.65393066e-04,  9.61914062e-02,
        1.68945312e-01,  2.60925293e-03,  8.93554688e-02,  6.49414062e-02,
        3.56445312e-02, -6.93359375e-02, -1.46484375e-01, -1.21093750e-01,
       -2.27539062e-01,  2.45361328e-02, -1.24511719e-01, -3.18359375e-01,
       -2.20703125e-01,  1.30859375e-01,  3.66210938e-02, -3.63769531e-02,
       -1.13281250e-01,  1.95312500e-01,  9.76562500e-02,  1.26953125e-01,
        6.59179688e-02,  6.93359375e-02,  1.02539062e-02,  1.75781250e-01,
       -1.68945312e-01,  1.21307373e-03, -2.98828125e-01, -1.15234375e-01,
        5.66406250e-02, -

In [27]:
model.wv.most_similar('tasty')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('delicious', 0.8730389475822449),
 ('scrumptious', 0.8007042407989502),
 ('yummy', 0.7856924533843994),
 ('flavorful', 0.7420164346694946),
 ('delectable', 0.7385422587394714),
 ('juicy_flavorful', 0.7114803791046143),
 ('appetizing', 0.7017216682434082),
 ('crunchy_salty', 0.7012300491333008),
 ('flavourful', 0.6912213563919067),
 ('flavoursome', 0.6857702732086182)]

In [29]:
model.wv.similarity('tasty','tast')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


0.44035053

In [30]:
import gensim
i=0
list_of_sent=[]
for sent in final['Text'].values:
    filtered_sentence=[]
    sent=cleanhtml(sent)
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):    
                filtered_sentence.append(cleaned_words.lower())
            else:
                continue 
    list_of_sent.append(filtered_sentence)

In [31]:
w2v_model=gensim.models.Word2Vec(list_of_sent,min_count=5,size=50, workers=4)

In [32]:
words = list(w2v_model.wv.vocab)
print(len(words))

34022


In [33]:
w2v_model.wv.most_similar('like')

[('resemble', 0.6845455765724182),
 ('prefer', 0.672731876373291),
 ('dislike', 0.6563374996185303),
 ('likebr', 0.6197497844696045),
 ('fake', 0.5972792506217957),
 ('think', 0.597212553024292),
 ('mean', 0.591109037399292),
 ('enjoy', 0.5899633169174194),
 ('love', 0.581458330154419),
 ('gross', 0.5805165767669678)]

In [34]:
sent_vectors = []; 
for sent in list_of_sent:
    sent_vec = np.zeros(50) 
    cnt_words =0; 
    for word in sent: 
        try:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
        except:
            pass
    sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


364171
50


In [ ]:


tfidf_feat = tf_idf_vect.get_feature_names() 
tfidf_sent_vectors = [];
row=0;
for sent in list_of_sent: 
    sent_vec = np.zeros(50) 
    weight_sum =0; 
    for word in sent: 
        try:
            vec = w2v_model.wv[word]
           
            tfidf = final_tf_idf[row, tfidf_feat.index(word)]
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
        except:
            pass
    sent_vec /= weight_sum
    tfidf_sent_vectors.append(sent_vec)
    row += 1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide
